In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


C:\Users\utilisateur\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning:

Columns (1,18) have mixed types.Specify dtype option on import or set low_memory=False.



,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
199,AFG,Asia,Afghanistan,2020-07-27,36263.0,106.0,1269.0,10.0,931.532,2.723,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
340,ALB,Europe,Albania,2020-07-27,4763.0,126.0,138.0,4.0,1655.084,43.783,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
545,DZA,Africa,Algeria,2020-07-27,27357.0,593.0,1155.0,9.0,623.862,13.523,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-07-26,France,181625.0,0.0,30210.0,0.0,0.0,0.0
0,2020-07-26,China,86525.0,0.0,4660.0,0.0,0.0,0.0
0,2020-07-26,Italy,246227.0,0.0,35131.0,0.0,0.0,0.0
0,2020-07-26,Spain,274502.0,0.0,28445.0,0.0,0.0,0.0
0,2020-07-26,United States,4249892.0,0.0,147748.0,0.0,0.0,0.0
0,2020-07-26,World,16297621.0,0.0,651776.0,0.0,0.0,0.0
0,2020-07-26,United Kingdom,299697.0,0.0,45897.0,0.0,0.0,0.0
0,2020-07-26,Germany,206060.0,0.0,9132.0,0.0,0.0,0.0
0,2020-07-26,Iran,288112.0,0.0,15216.0,0.0,0.0,0.0
0,2020-07-26,Turkey,226067.0,0.0,5612.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-07-27,France,181625.0,180528.0,30210.0,30192.0,1097.0,18.0
0,2020-07-27,China,86525.0,86570.0,4660.0,4652.0,-45.0,8.0
0,2020-07-27,Italy,246227.0,246118.0,35131.0,35107.0,109.0,24.0
0,2020-07-26,Spain,274502.0,272421.0,28445.0,28432.0,2081.0,13.0
0,2020-07-27,United States,4249892.0,4234020.0,147748.0,146935.0,15872.0,813.0
0,2020-07-27,World,16297621.0,16249165.0,651776.0,649208.0,48456.0,2568.0
0,2020-07-27,United Kingdom,299697.0,299426.0,45897.0,45752.0,271.0,145.0
0,2020-07-27,Germany,206060.0,205609.0,9132.0,9118.0,451.0,14.0
0,2020-07-27,Iran,288112.0,291172.0,15216.0,15700.0,-3060.0,-484.0
0,2020-07-27,Turkey,226067.0,226100.0,5612.0,5613.0,-33.0,-1.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")